In [3]:
import os
from itertools import combinations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup

In [42]:
# https://ictv.global/report/chapter/geminiviridae/geminiviridae/begomovirus

In [ ]:
# CDS -> Codon DNA Segments.

In [ ]:
# HTML parsing logic.

In [ ]:
# Copy pasted the HTML manually from Firefox. Fuck lazy loads.

with open("./data/Genus Begomovirus ICTV.html", mode = "r", encoding = "utf8") as html:
    content = html.read()
    soup = BeautifulSoup(content, "html.parser")

table = soup.find_all("table", attrs = {"class" : "virus-isolates-table"})[0]

headers = [th.text for th in table.find_all("th")]
headers

table_dict = {
    "Species": [],
    "Virus name": [],
    "Isolate": [],
    "Accession": [],
    "Available sequence": [],
    "Abbrev":[]
}

# Except for the row with table headers, there seems to be two kinds of table rows.
# 1) <tr class="virus-row">
# 2) <tr class="alt-virus-row">

counter = 0
for tr in table.find_all("tr"):
    if counter == 0:
        counter += 1
        continue
    else:
        table_dict["Species"].append(tr.find("td", attrs = {"class": "col-species"}).text)
        table_dict["Virus name"].append(tr.find("td", attrs = {"class": "col-alternativeNameCSV"}).text)
        table_dict["Isolate"].append(tr.find("td", attrs = {"class": "col-isolate"}).text)
        table_dict["Accession"].append(tr.find("td", attrs = {"class": "col-accessionNumber"}).text)
        table_dict["Available sequence"].append(tr.find("td", attrs = {"class": "col-availableSequence"}).text)
        table_dict["Abbrev"].append(tr.find("td", attrs = {"class": "col-abbrev"}).text)
        counter += 1

begomovirus = pd.DataFrame(table_dict)
begomovirus.to_csv("./data/begomovirus_ictv.csv", sep = ',', header = True, index = False, mode = "w")

In [ ]:
# Cassava associated viruses - filtering logic.

In [ ]:
ictv_begomo = pd.read_csv("./data/begomovirus_ictv.csv")
ind = np.array([True if "cassava" in name.lower() else False for name in ictv_begomo["Virus name"].values])
ictv_begomo.loc[ind, :].to_csv("./data/cassava_filtered.csv", sep = ',', header = True, index = False, mode = "w")

In [3]:
# https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=fasta&id=89330928&conwithfeat=on&withparts=on&show-sequence=on&hide-cdd=on&ncbi_phid=CE8AC93650A0D641000000000A89088C

In [7]:
cmv = pd.read_csv("./data/meta/cassava_filtered.csv")

In [8]:
[acc[0] for acc in cmv.Accession.str.split(";")]

['DNA-A: HE616777',
 'DNA-A: J02057',
 'DNA-A: HE617299',
 'DNA-A: AF112354',
 'DNA-A: AJ717572',
 'DNA-A: AJ006460',
 'DNA-A: AF126806',
 'DNA-A: AJ717542',
 'JX473582',
 'DNA-A: AY795983',
 'DNA-A: AF422174',
 'DNA-A: Z24758',
 'DNA-A: JF496657',
 'DNA-A: GQ924760',
 'DNA-A: AJ575819',
 'DNA-A: AF155806',
 'DNA-A: AJ314737',
 'DNA-A: AJ579307']

In [9]:
accessions = []

for fasta in os.listdir("./data/refseqs/edited/"):
    with open("./data/refseqs/edited/" + fasta, "r") as file:
        accessions.append(str(file.read()).split("|")[0].replace('>', ''))

In [10]:
accessions

['HE616777',
 'J02057',
 'HE617299',
 'AF112354',
 'AJ717572',
 'AJ006460',
 'AJ717542',
 'JX473582',
 'AY795983',
 'AF126806',
 'AF422174',
 'Z24758',
 'GQ924760',
 'AJ575819',
 'JF496657',
 'AF155806',
 'AJ579307',
 'AJ314737']

In [11]:
cmv.head()

,Species,Virus name,Isolate,Accession,Available sequence,Abbrev
0,African cassava mosaic Burkina Faso virus,African cassava mosaic Burkina Faso virus,BF-Oua-127-08,DNA-A: HE616777;DNA-B: HE616778,Complete genome,ACMBFV
1,African cassava mosaic virus,African cassava mosaic virus,Cameroon/1998,DNA-A: J02057;DNA-B: J02058,Complete genome,ACMV
2,Cassava mosaic Madagascar virus,cassava mosaic Madagascar virus,Madgascar/Toliary/2006,DNA-A: HE617299;DNA-B: HE617300,Complete genome,CMMGV
3,East African cassava mosaic Cameroon virus,East African cassava mosaic Cameroon virus,Cameroon/1998,DNA-A: AF112354;DNA-B: AF112355,Complete genome,EACMCMV
4,East African cassava mosaic Kenya virus,East African cassava mosaic Kenya virus,Kenya/Mitaboni/K298/2002,DNA-A: AJ717572;DNA-B: AJ704971,Complete genome,EACMKV


In [12]:
identifiers = []

for fasta in os.listdir("./data/refseqs/edited/"):
    with open("./data/refseqs/edited/" + fasta, "r") as fp:
        identifiers.append(str(fp.read()).splitlines()[0].replace('>', ''))

In [13]:
identifiers

['HE616777|ACMBFV(BF:Oua:BF127:08)',
 'J02057|ACMV(KE:West Kenyan 844:98)',
 'HE617299|CMMGV(MG:Tol:06)',
 'AF112354|EACMCMV(CMRN:WACMV/CM:98)',
 'AJ717572|EACMKV(KE:EACMKV-[K298]:02)',
 'AJ006460|EACMMV(MW:MK:96)',
 'AJ717542|EACMV/KE2(KE:EACMV-KE2[K48]:01)',
 'JX473582|EACMV/MW(MW:Malawi/2007/8N:07)',
 'AY795983|EACMV/TZ(TZ:TZ1:01)',
 'AF126806|EACMV/UG(UG:EACMV/Ug2//Svr:97)',
 'AF422174|EACMZV(TZ:ZNZ:Uguja:98)',
 'Z24758|ICMV/IN(IN:Triv:86)',
 'GQ924760|ICMV/Jat(IN:Dharwad:08)',
 'AJ575819|ICMV/Ker(IN:KL:Adivaram 2:02)',
 'JF496657|ICMV(IN:MH:Jalgaon:09)',
 'AF155806|SACMV(ZA:01)',
 'AJ579307|SLCMV/IN(IN:KL:Adivaram:03)',
 'AJ314737|SLCMV/LK(LK:CMB:SLCMV-Col:98)']

In [14]:
IDENTS = [
"ACMBFV(BF:Oua:BF127:08)",
"ACMV(KE:West Kenyan 844:98)",
"CMMGV(MG:Tol:06)",
"EACMCMV(CMRN:WACMV/CM:98)",
"EACMKV(KE:EACMKV-[K298]:02)",
"EACMMV(MW:MK:96)",
"EACMV/KE2(KE:EACMV-KE2[K48]:01)",
"EACMV/MW(MW:Malawi/2007/8N:07)",
"EACMV/TZ(TZ:TZ1:01)",
"EACMV/UG(UG:EACMV/Ug2//Svr:97)",
"EACMZV(TZ:ZNZ:Uguja:98)",
"ICMV/IN(IN:Triv:86)",
"ICMV(IN:MH:Jalgaon:09)",
"ICMV/Jat(IN:Dharwad:08)",
"ICMV/Ker(IN:KL:Adivaram 2:02)",
"SACMV(ZA:01)",
"SLCMV/IN(IN:KL:Adivaram:03)",
"SLCMV/LK(LK:CMB:SLCMV-Col:98)"
]

In [17]:
# Good :)

[ident.split('|')[1] in IDENTS for ident in identifiers]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [26]:
# Now we need to write new identifiers.

sl_viruses = list(filter(lambda fname: fname if ".fasta" in fname else None, os.listdir("./data/srilanka/")))
sl_viruses

['AF314144.fasta',
 'GU323321.fasta',
 'JN555600.fasta',
 'KX698088.fasta',
 'KX698089.fasta',
 'KX698090.fasta',
 'KX698091.fasta',
 'KX698092.fasta',
 'KX698093.fasta',
 'MH455207.fasta',
 'MH455208.fasta',
 'MH455209.fasta',
 'MH455210.fasta',
 'MH455211.fasta',
 'MH455212.fasta',
 'MN384976.fasta',
 'MN389529.fasta',
 'MT572475.fasta',
 'MT572476.fasta',
 'MT572477.fasta',
 'NC_004647.fasta',
 'NC_031754.fasta',
 'NC_055131.fasta']

In [28]:
fastas_read = []

for fname in sl_viruses:
    with open("./data/srilanka/" + fname, "r") as fp:
        fastas_read.append(str(fp.read()))

In [49]:
accessions = [
"AF314144",
"NC_055131",
"JN555600",
"GU323321",
"KX698088",
"KX698091",
"KX698092",
"MN384976",
"KX698090",
"MH455208",
"MN389529",
"KX698093",
"NC_031754",
"MH455207",
"MH455212",
"MT572476",
"MH455210",
"MT572475",
"MT572477",
"MH455211",
"KX698089",
"MH455209",
"NC_004647"
]

NW_IDENTS = [
"AYVSLV(LK:99)",
"ChiLCV(LK:Noch:CL-14:09)",
"ChiLCV(LK:Noch:CL-15:09)",
"HgYMV(LK:09)",
"OELuV(LK:Jaf:15)",
"OELuV(LK:Put:15)",
"OELuV(LK:Mat:15)",
"OELCuV(LK:DAJfn01:18)",
"OELuV(LK:Vav:15)",
"OELCuV(LK:Va02:15)",
"OELCuV(LK:DAVav02:18)",
"OELCuV(LK:Bat:15)",
"OELuV(LK:Tri:15)",
"OELCuV(LK:Tr02:15)",
"BYVMV(LK:Ma02:15)",
"BYVMV(LK:MI:Part:XX)",
"BYVMV(LK:Ka02:15)",
"BYVMV(LK:Gan:Part:XX)",
"BYVMV(LK:Ang:Part:XX)",
"BYVMV(LK:Pu02:15)",
"BYVMV(LK:Kan:15)",
"BYVMV(LK:Ja02:15)",
"ToLCLKV(LK:Bad:Band:97)"
]

In [52]:
# Good :)

len(NW_IDENTS), len(accessions)

(23, 23)

In [56]:
lookup = {acc: ident for acc, ident in zip(accessions, NW_IDENTS)}

In [57]:
lookup

{'AF314144': 'AYVSLV(LK:99)',
 'NC_055131': 'ChiLCV(LK:Noch:CL-14:09)',
 'JN555600': 'ChiLCV(LK:Noch:CL-15:09)',
 'GU323321': 'HgYMV(LK:09)',
 'KX698088': 'OELuV(LK:Jaf:15)',
 'KX698091': 'OELuV(LK:Put:15)',
 'KX698092': 'OELuV(LK:Mat:15)',
 'MN384976': 'OELCuV(LK:DAJfn01:18)',
 'KX698090': 'OELuV(LK:Vav:15)',
 'MH455208': 'OELCuV(LK:Va02:15)',
 'MN389529': 'OELCuV(LK:DAVav02:18)',
 'KX698093': 'OELCuV(LK:Bat:15)',
 'NC_031754': 'OELuV(LK:Tri:15)',
 'MH455207': 'OELCuV(LK:Tr02:15)',
 'MH455212': 'BYVMV(LK:Ma02:15)',
 'MT572476': 'BYVMV(LK:MI:Part:XX)',
 'MH455210': 'BYVMV(LK:Ka02:15)',
 'MT572475': 'BYVMV(LK:Gan:Part:XX)',
 'MT572477': 'BYVMV(LK:Ang:Part:XX)',
 'MH455211': 'BYVMV(LK:Pu02:15)',
 'KX698089': 'BYVMV(LK:Kan:15)',
 'MH455209': 'BYVMV(LK:Ja02:15)',
 'NC_004647': 'ToLCLKV(LK:Bad:Band:97)'}

In [70]:
write_contents = []

for content in fastas_read:
    acc = content.splitlines()[0].split(" ")[0].replace(".1", '').replace('>', '')
    except_firstline = content.splitlines()[1:]
    nw_firstline = '>' + acc + '|' + lookup.get(acc, "Ooops")
    write_contents.append(nw_firstline + '\n' + "\n".join(except_firstline))

In [72]:
len(sl_viruses), len(write_contents)

(23, 23)

In [74]:
for i in range(len(sl_viruses)):
    with open("./data/srilanka/edited/" + sl_viruses[i], "w+") as fp:
        fp.write(write_contents[i])

In [89]:
# Test the writes are okay!

old = list(filter(lambda fname: fname if ".fasta" in fname else None, os.listdir("./data/srilanka/")))
new = list(filter(lambda fname: fname if ".fasta" in fname else None, os.listdir("./data/srilanka/edited/")))

In [92]:
sum([o in new for o in old])

23

In [105]:
for fo, fn in zip(old, new):
    with open("./data/srilanka/" + fo, "r") as fpo:
        oarr = np.array(list("".join(str(fpo.read()).splitlines()[1:])))
    with open("./data/srilanka/edited/" + fo, "r") as fpn:
        narr = np.array(list("".join(str(fpn.read()).splitlines()[1:])))
    chk_status = oarr == narr
    print(f"{chk_status.mean():.1f}, {chk_status.size: 5d}")

1.0,  2748
1.0,  2735
1.0,  2754
1.0,  2738
1.0,  2786
1.0,  2738
1.0,  2738
1.0,  2738
1.0,  2738
1.0,  2762
1.0,  2738
1.0,  2749
1.0,  2743
1.0,  2735
1.0,  2741
1.0,  2738
1.0,  2741
1.0,   656
1.0,   656
1.0,   657
1.0,  2756
1.0,  2738
1.0,  2754


In [107]:
# Woohoo motherfuckers! The writes are in fucking fact okay!